# 1. 파일 Import / 벡터 DB 저장

In [1]:
import pandas as pd
import numpy as np
import os
import torch
from keybert import KeyBERT
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

from langchain_core.messages import ChatMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.embeddings import Embeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.document_loaders import TextLoader, PyPDFLoader, PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from langchain_openai import ChatOpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langserve.pydantic_v1 import BaseModel, Field
from typing import List, Union

from openai import OpenAI
import os
import warnings

warnings.filterwarnings("ignore")

import os
import shutil
import gc
import warnings
import torch
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import PreTrainedTokenizerFast

warnings.filterwarnings("ignore")

# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

/Users/shinminseog/opt/anaconda3/envs/dl/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Using device: mps


In [ ]:
# PDF 파일이 있는 디렉토리
pdf_dir = "file/현행법령/"

# 벡터 저장소 경로
vectorstore_path = 'file/law_vectorstore_3'

# 기존 Chroma 벡터 저장소 삭제 (처음부터 새로 저장하기 위해)
if os.path.exists(vectorstore_path):
    print("Removing existing Chroma vectorstore...")
    shutil.rmtree(vectorstore_path)

# 새로 저장소 디렉토리 생성
os.makedirs(vectorstore_path, exist_ok=True)

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        # SentenceTransformer 모델을 MPS 디바이스로 이동
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 임베딩을 GPU에서 처리
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# 임베딩 모델 설정
embedding_model = MyEmbeddings()

# 새로 Chroma 벡터 저장소 생성
print("Creating new Chroma vectorstore...")
vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embedding_model)

# PDF 파일들을 읽어 Chroma 벡터 저장소에 추가
for file_name in os.listdir(pdf_dir):
    if file_name.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, file_name)

        # PDF 파일 로더로 파일 읽기
        loader = PyMuPDFLoader(pdf_path)
        document = loader.load()

        # 텍스트 분할
        docs_split = splitter.split_documents(document)

        # 분할된 문서를 벡터 저장소에 추가
        vectorstore.add_documents(docs_split)
        
        # 벡터 저장소 업데이트
        vectorstore.persist()

        # 대용량 객체 메모리 해제
        del document, docs_split
        gc.collect()

        # 파일 처리 완료 메시지 출력
        print(f"Processing and embedding: {file_name}")

print("All documents processed and Chroma Vectorstore created successfully.")

In [2]:
# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        # SentenceTransformer 모델을 MPS 디바이스로 이동
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 임베딩을 GPU에서 처리
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# 임베딩 모델 설정
embedding_model = MyEmbeddings()
vectorstore_path = 'file/law_vectorstore_2'

In [17]:
vs_test = Chroma(persist_directory=vectorstore_path, embedding_function=embedding_model)
retriever = vs_test.as_retriever(search_kwargs={'k': 30})

In [16]:
r_test1 = retriever.get_relevant_documents("민법에서 대리권이 소멸되는 경우는 무엇인가요?")
r_test2 = retriever.get_relevant_documents("도로교통법에서 음주운전 기준은 어떻게 되나요?")

In [ ]:
num = 1
for i in r_test1:
    print(num, i.page_content)
    num += 1

In [ ]:
retriever.get_relevant_documents("도로교통법에서 음주운전 기준은 어떻게 되나요?")

In [ ]:
num = 1
for i in r_test2:
    print(num, i.page_content)
    num += 1

In [ ]:
score_df = pd.read_csv("file/현행법령_질문_타겟답변.csv")
score_df

In [ ]:
retriever.get_relevant_documents("민법에서 대리권이 소멸되는 경우는 무엇인가요?")

In [9]:
# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# PDF 파일이 있는 디렉토리
pdf_dir = "file/"

# 토크나이저 초기화
tokenizer_2 = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 텍스트 분할기
splitter_2 = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    # separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer_2.encode(text)),
)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        # SentenceTransformer 모델을 MPS 디바이스로 이동
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 임베딩을 GPU에서 처리
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# 임베딩 모델 설정
embedding_model = MyEmbeddings()
vectorstore_2 = Chroma(embedding_function=embedding_model)

file_name = "최신 소방 관계 법령.pdf"

pdf_path = pdf_dir + file_name

# PDF 파일 로더로 파일 읽기
loader = PyMuPDFLoader(pdf_path)
document = loader.load()

# 텍스트 분할
docs_split = splitter_2.split_documents(document)

# 분할된 문서를 벡터 저장소에 추가
vectorstore_2.add_documents(docs_split)

print("All documents processed and Chroma Vectorstore created successfully.")

Using device: mps
All documents processed and Chroma Vectorstore created successfully.


In [26]:
import re
import torch
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from sentence_transformers import SentenceTransformer
from typing import List
from langchain.schema import Document  # Document 객체를 가져옵니다

# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-sroberta-base-continue-learning-by-mnr')

def preprocess_text(text: str) -> str:
    # 여러 개의 공백을 하나로 줄이기
    text = re.sub(r'\s+', ' ', text)
    
    # 불필요한 페이지 번호나 공백을 제거
    text = re.sub(r'\b\d+\b', '', text)  # 숫자(페이지 번호) 제거
    
    # 특정 키워드 제거 (법제처, 국가법령정보센터 등)
    keywords_to_remove = ["법제처", "국가법령정보센터"]
    for keyword in keywords_to_remove:
        text = text.replace(keyword, "")
    
    # 텍스트 양쪽의 불필요한 공백 제거
    text = text.strip()
    
    # 특수문자 제거
    text = re.sub(r'[^A-Za-z0-9가-힣\s.,]', '', text)

    return text

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.cpu().tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# 임베딩 모델 설정
embedding_model = MyEmbeddings()

# Chroma 클라이언트 초기화 (임베딩 함수 설정)
client = Chroma(embedding_function=embedding_model)

file_name = "최신 소방 관계 법령.pdf"

# PDF 파일 로더로 파일 읽기
loader = PyMuPDFLoader('file/' + file_name)
document = loader.load()

# 텍스트 전처리 적용
preprocessed_docs = [Document(page_content=preprocess_text(doc.page_content), metadata=doc.metadata) for doc in document]

# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=768, 
    chunk_overlap=76,
    # separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)

# 전처리된 텍스트를 분할
docs_split = splitter.split_documents(preprocessed_docs)

# 분할된 문서를 벡터 저장소에 추가
client.add_documents(docs_split)

print("All documents processed and Chroma Vectorstore created successfully.")

Using device: mps
All documents processed and Chroma Vectorstore created successfully.


In [12]:
# 5. 유사도 점수를 포함한 검색
retriever_with_score = client.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.7
    }
)

In [3]:
retriever = client.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.0000001,  # 최소 유사도 점수 설정
        "k": 5  # 반환할 최대 문서 수
    }
)

In [26]:
retriever_test = client.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={
        'k': 5,
        'score_threshold' : 0,
    },    
)

ValidationError: 1 validation error for VectorStoreRetriever
__root__
  `score_threshold` is not specified with a float value(0~1) in `search_kwargs`. (type=value_error)

In [34]:
retriever = client.as_retriever(search_kwargs={'k': 5})

In [4]:
retriever.get_relevant_documents("소화기 설치 조건")

No relevant docs were retrieved using the relevance score threshold 1e-07


[]

In [13]:
retriever_with_score.get_relevant_documents("소화기 설치 조건")

TypeError: query() got an unexpected keyword argument 'score_threshold'

In [27]:
retriever_with_score = client.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}  # 임계값 필터링 전에 충분한 결과를 가져오기 위해 k를 증가시킵니다
)

query = "소화기 설치 조건"
docs = retriever_with_score.get_relevant_documents(query)

# 결과를 후처리하여 임계값을 적용합니다
threshold = 0
filtered_docs = [doc for doc in docs if doc.metadata.get('score', 0) >= threshold]

for doc in filtered_docs:
    print(f"Score: {doc.metadata.get('score', 0)}")
    print(f"Content: {doc.page_content}")
    print("-" * 80)

Score: 0
Content: 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 이하의 높이에 설치할 것 . 소화기의 상부에 소화기라고 표시한 조명식 또는 반사식의 표지판을 부착하여 사용자가 쉽게 알 수
--------------------------------------------------------------------------------
Score: 0
Content: . 소화기란 소화기구 및 자동소화장치의 화재안전성능기준 NF PC101  제3조제2호에 서 정의하는 소화기를 말한다. . 간이소화장치란 건설현장에서 화재발생 시 신속한 화재 진압이 가능하도록 물을 방수하 는 형태의 소화장치를 말한다. . 비상경보장치란 발신기, 경종, 표시등 및 시각경보장치가 결합된 형태의 것으로서
--------------------------------------------------------------------------------
Score: 0
Content: 시까지 작업지점으로부터  미 터 이내의 쉽게 보이는 장소에 능력단위 3단위 이상인 소화기 2개 이상과 대형소화기 1개 이상을 추가 배치해야 한다. . 소화기라고 표시한 축광식 표지를 소화기 설치장소 보기 쉬운 곳에 부착하여야 한다. 제6조간이소화장치의 성능 및 설치기준 간이소화장치의 성능 및 설치기준은
--------------------------------------------------------------------------------
Score: 0
Content: 사용 및 운반의 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 이하의 높이에 설치할 것 . 소화기의 상부에 소화기라고 표시한 조명식 또는 반사식의 표지판을 부착하여 사용자

In [28]:
query = "소화기 설치 조건"
docs_and_scores = client.similarity_search_with_relevance_scores(query, k=5)

print("검색 결과:")
for doc, score in docs_and_scores:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content[:100]}...")  # 내용의 일부만 출력
    print("-" * 50)

검색 결과:
Score: -94.04268900129463
Content: 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 ...
--------------------------------------------------
Score: -94.68485322089728
Content: . 소화기란 소화기구 및 자동소화장치의 화재안전성능기준 NF PC101  제3조제2호에 서 정의하는 소화기를 말한다. . 간이소화장치란 건설현장에서 화재발생 시 신속한 화재 진압이...
--------------------------------------------------
Score: -95.19160804560414
Content: 시까지 작업지점으로부터  미 터 이내의 쉽게 보이는 장소에 능력단위 3단위 이상인 소화기 2개 이상과 대형소화기 1개 이상을 추가 배치해야 한다. . 소화기라고 표시한 축광식 표지...
--------------------------------------------------
Score: -98.08723275691695
Content: 사용 및 운반의 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으...
--------------------------------------------------
Score: -98.65587668832677
Content: 건설현장의 화재안전성능기준 NF PC   제4조다른 화재안전성능기준과의 관계 건설현장의 임시소방시설 설치 및 관리와 관련하여 이 기준에서 정하지 않은 사항은 개별 화재안전성능기준을...
--------------------------------------------------


In [29]:
query = "소화기 설치 조건"
docs_and_scores = client.similarity_search_with_relevance_scores(query, k=10)

print("검색 결과 및 점수:")
for doc, score in docs_and_scores:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content[:100]}...")
    print("-" * 50)

# 점수 통계 분석
scores = [score for _, score in docs_and_scores]
print(f"최소 점수: {min(scores)}")
print(f"최대 점수: {max(scores)}")
print(f"평균 점수: {sum(scores) / len(scores)}")

검색 결과 및 점수:
Score: -94.04268900129463
Content: 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 ...
--------------------------------------------------
Score: -94.68485322089728
Content: . 소화기란 소화기구 및 자동소화장치의 화재안전성능기준 NF PC101  제3조제2호에 서 정의하는 소화기를 말한다. . 간이소화장치란 건설현장에서 화재발생 시 신속한 화재 진압이...
--------------------------------------------------
Score: -95.19160804560414
Content: 시까지 작업지점으로부터  미 터 이내의 쉽게 보이는 장소에 능력단위 3단위 이상인 소화기 2개 이상과 대형소화기 1개 이상을 추가 배치해야 한다. . 소화기라고 표시한 축광식 표지...
--------------------------------------------------
Score: -98.08723275691695
Content: 사용 및 운반의 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으...
--------------------------------------------------
Score: -98.65587668832677
Content: 건설현장의 화재안전성능기준 NF PC   제4조다른 화재안전성능기준과의 관계 건설현장의 임시소방시설 설치 및 관리와 관련하여 이 기준에서 정하지 않은 사항은 개별 화재안전성능기준을...
--------------------------------------------------
Score: -98.88497212114132
Content: 소화기

In [31]:
def normalize_scores(docs_and_scores):
    scores = [score for _, score in docs_and_scores]
    min_score = min(scores)
    max_score = max(scores)
    
    if min_score == max_score:
        return [(doc, 1.0) for doc, _ in docs_and_scores]
    
    normalized = []
    for doc, score in docs_and_scores:
        normalized_score = (score - min_score) / (max_score - min_score)
        normalized.append((doc, normalized_score))
    
    return normalized

normalized_docs_and_scores = normalize_scores(docs_and_scores)

print("정규화된 검색 결과 및 점수:")
for doc, score in normalized_docs_and_scores:
    print(f"Normalized Score: {score}")
    print(f"Content: {doc.page_content[:100]}...")
    print("-" * 50)

정규화된 검색 결과 및 점수:
Normalized Score: 1.0
Content: 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 ...
--------------------------------------------------
Normalized Score: 0.9152718995169754
Content: . 소화기란 소화기구 및 자동소화장치의 화재안전성능기준 NF PC101  제3조제2호에 서 정의하는 소화기를 말한다. . 간이소화장치란 건설현장에서 화재발생 시 신속한 화재 진압이...
--------------------------------------------------
Normalized Score: 0.8484099156235282
Content: 시까지 작업지점으로부터  미 터 이내의 쉽게 보이는 장소에 능력단위 3단위 이상인 소화기 2개 이상과 대형소화기 1개 이상을 추가 배치해야 한다. . 소화기라고 표시한 축광식 표지...
--------------------------------------------------
Normalized Score: 0.4663568924061184
Content: 사용 및 운반의 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으...
--------------------------------------------------
Normalized Score: 0.39132916789451905
Content: 건설현장의 화재안전성능기준 NF PC   제4조다른 화재안전성능기준과의 관계 건설현장의 임시소방시설 설치 및 관리와 관련하여 이 기준에서 정하지 않은 사항은 개별 화재안전성능기준을...
-------------------------------------------

In [21]:
# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.cpu().tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]


embeddings = MyEmbeddings()
test_text = "테스트 문장입니다."
embedding_vector = embeddings.embed_query(test_text)

print(f"임베딩 벡터의 길이: {len(embedding_vector)}")
print(f"임베딩 벡터의 일부: {embedding_vector[:5]}...")

임베딩 벡터의 길이: 768
임베딩 벡터의 일부: [-0.5610249638557434, -0.6634182929992676, -0.7814193367958069, -0.43886685371398926, -0.4771650731563568]...


In [32]:
docs = client.similarity_search(query, k=5)

print("기본 유사도 검색 결과:")
for doc in docs:
    print(f"Content: {doc.page_content[:100]}...")
    print("-" * 50)

기본 유사도 검색 결과:
Content: 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 ...
--------------------------------------------------
Content: . 소화기란 소화기구 및 자동소화장치의 화재안전성능기준 NF PC101  제3조제2호에 서 정의하는 소화기를 말한다. . 간이소화장치란 건설현장에서 화재발생 시 신속한 화재 진압이...
--------------------------------------------------
Content: 시까지 작업지점으로부터  미 터 이내의 쉽게 보이는 장소에 능력단위 3단위 이상인 소화기 2개 이상과 대형소화기 1개 이상을 추가 배치해야 한다. . 소화기라고 표시한 축광식 표지...
--------------------------------------------------
Content: 사용 및 운반의 편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으...
--------------------------------------------------
Content: 건설현장의 화재안전성능기준 NF PC   제4조다른 화재안전성능기준과의 관계 건설현장의 임시소방시설 설치 및 관리와 관련하여 이 기준에서 정하지 않은 사항은 개별 화재안전성능기준을...
--------------------------------------------------


In [35]:
retriever.get_relevant_documents("소화기 설치 기준")

[Document(metadata={'author': '', 'creationDate': "D:20240821042234+09'00'", 'creator': '', 'file_path': 'file/최신 소방 관계 법령.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20240821042234+09'00'", 'page': 925, 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'file/최신 소방 관계 법령.pdf', 'subject': '', 'title': '', 'total_pages': 2337, 'trapped': ''}, page_content='편리성을 고려하여 7킬로그램 이하로 할 것 . 소화기는 사람이 출입할 수 있는 출입구 환기구, 작업구를 포함한다 부근에 5개 이상 설치 할 것 . 소화기는 바닥면으로부터 .5미터 이하의 높이에 설치할 것 . 소화기의 상부에 소화기라고 표시한 조명식 또는 반사식의 표지판을 부착하여 사용자가 쉽게 알 수'),
 Document(metadata={'author': '', 'creationDate': "D:20240821042234+09'00'", 'creator': '', 'file_path': 'file/최신 소방 관계 법령.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20240821042234+09'00'", 'page': 780, 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'file/최신 소방 관계 법령.pdf', 'subject': '', 'title': '', 'total_pages': 2337, 'trapped': ''}, page_content='보행로를 말한다. 이하 같다 으로부터 .5미터 이하의 높이에 설치할 것 . 소화기구함의 상부에 소화기라고 조명식 또는 반사식의 표지판을 부착하여 사용자가 쉽 게 인지할 수 

In [32]:
query = "소화기"
results = retriever.get_relevant_documents(query)

for doc in results:
    print(f"Content: {doc.page_content}")
    print(f"Score: {doc.metadata['score']}")
    print("-" * 50)

No relevant docs were retrieved using the relevance score threshold 1e-07


In [ ]:
from transformers import AutoModel, AutoTokenizer

# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-sroberta-base-continue-learning-by-mnr')
# tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")

# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr", device=device):
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True).tolist()
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]
# 임베딩 모델 설정
embedding_model = MyEmbeddings()
vectorstore = Chroma(embedding_function=embedding_model)

file_name = "최신 소방 관계 법령.pdf"

# PDF 파일 로더로 파일 읽기
loader = PyMuPDFLoader('file/' + file_name)
document = loader.load()

# 텍스트 분할
docs_split = splitter.split_documents(document)

# 분할된 문서를 벡터 저장소에 추가
vectorstore.add_documents(docs_split)

print("All documents processed and Chroma Vectorstore created successfully.")

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [ ]:
retriever.get_relevant_documents("소방법에서 소화기란 어떤것인가요?")

In [ ]:
# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# PDF 파일이 있는 디렉토리
pdf_dir = "file/현행법령/"

# 벡터 저장소 경로
vectorstore_path = 'file/law_vectorstore_test'

# 기존 Chroma 벡터 저장소 삭제 (처음부터 새로 저장하기 위해)
if os.path.exists(vectorstore_path):
    print("Removing existing Chroma vectorstore...")
    shutil.rmtree(vectorstore_path)

# 새로 저장소 디렉토리 생성
os.makedirs(vectorstore_path, exist_ok=True)

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=256, 
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)


# 새로 Chroma 벡터 저장소 생성
print("Creating new Chroma vectorstore...")
vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embedding_model)

file_name = "도로교통법(법률)(제19745호)(20241025).pdf"

pdf_path = pdf_dir + file_name

# PDF 파일 로더로 파일 읽기
loader = PyMuPDFLoader(pdf_path)
document = loader.load()

# 텍스트 분할
docs_split = splitter.split_documents(document)

# 분할된 문서를 벡터 저장소에 추가
vectorstore.add_documents(docs_split)

# 벡터 저장소 업데이트
vectorstore.persist()

print("All documents processed and Chroma Vectorstore created successfully.")

In [ ]:
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.embeddings import Embeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from typing import List
from transformers import AutoTokenizer

# MPS(Apple Silicon GPU)가 사용 가능한지 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# PDF 파일이 있는 디렉토리
pdf_dir = "file/현행법령/"
file_name = "도로교통법(법률)(제19745호)(20241025).pdf"
pdf_path = pdf_dir + file_name

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=256, 
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True, device=device)
        return embeddings.cpu().tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# 임베딩 모델 설정
embedding_model = MyEmbeddings()
vectorstore = Chroma(embedding_function=embedding_model)

# PDF 파일 로더로 파일 읽기
loader = PyMuPDFLoader(pdf_path)
document = loader.load()

# 텍스트 분할
docs_split = splitter.split_documents(document)

# 분할된 문서를 벡터 저장소에 추가
vectorstore.add_documents(docs_split)

print("All documents processed and in-memory Chroma Vectorstore created successfully.")

# 예시 쿼리로 테스트
query = ""
results = vectorstore.similarity_search(query, k=3)

print("\nResults from the query:")
for doc in results:
    print(doc.page_content[:100] + "...")

In [5]:
retriever = vectorstore.as_retriever(k_wargs={'k' : 5})
result = vectorstore.similarity_search(query, k = 5)

In [ ]:
retriever.get_relevant_documents("음주 운전")[0].page_content